In [ ]:
import mne
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, LSTM, Dense, Flatten

# Set the path to the dataset
data_path = '/path/to/dataset'

# Download the dataset from PhysioNet
mne.datasets.physionet_motor_imagery.load_data(subjects=list(range(1, 110)))

# Loop over all subjects and tasks
for subject in range(1, 110):
    for task in range(1, 4):
        # Load the dataset using MNE
        raw = mne.io.read_raw_edf(os.path.join(data_path, f'S{subject:03}R{task:02}.edf'), preload=True)
        events, _ = mne.events_from_annotations(raw)
        epochs = mne.Epochs(raw, events, tmin=0, tmax=4, baseline=None, event_id={'left': 1, 'right': 2})

        # Get the data and labels from the epochs
        data = epochs.get_data()
        labels = epochs.events[:, -1] - 1

        # Reshape the data to fit the CNN-LSTM model
        data = data.reshape((data.shape[0], 1, data.shape[1], data.shape[2]))

        # Define the CNN-LSTM model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1, data.shape[2], data.shape[3])))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(LSTM(64, return_sequences=True))
        model.add(Dropout(0.5))
        model.add(Dense(2, activation='softmax'))

        # Compile the model
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)

        # Evaluate the model
        test_loss, test_acc = model.evaluate(data, labels)
        print(f'Subject {subject}, Task {task}: Test accuracy = {test_acc}')
        
        
